In [2]:
## 模型与剖分

import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

from tool import get_bb 


n   = 5  #剖分次数
Lam = [1,1e1,1e2,1e3,1e4,1e5]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
E_rel = np.zeros_like(E)

class PDE():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh
    
    @cartesian
    def source(self, p):
        x   = p[..., 0]
        y   = p[..., 1]
        mu  = self.mu
        lam = self.lam
        
        sin = np.sin
        cos = np.cos
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = -((2 * mu + lam) * y * (y - 1) * (2 * cos(x) - (x - 1) * sin(x))
                        + (mu + lam) * (2 * x - 1) * (sin(y) + (y - 1) * cos(y)) 
                        + 2 * mu * (x -1) * sin(x))
        val[..., 1] = -((2 * mu + lam) * x * (x - 1) * (2 * cos(y) - (y - 1) * sin(y))
                        + (mu + lam) * (2 * y - 1) * (sin(x) + (x - 1) * cos(x))
                        + 2 * mu * (y - 1) * sin(y))

        #val[..., 0] = -(2 * (2 * mu + lam) * y * (y - 1) 
        #                + (mu + lam) * (2 * x - 1) * (2 * y - 1)
        #                + 2 * mu * x * (x - 1))
        #val[..., 1] = -(2 * (2 * mu + lam) * x * (x - 1)
        #                + (mu + lam) * (2 * x - 1) * (2 * y - 1)
        #                + 2 * mu * y * (y - 1))
        return val
    
    def dirichlet(self, p):
        var = np.zeros_like(p)
        return var
    
    def solution(self, p):
        x = p[..., 0]
        y = p[..., 1]
        
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = y * (x - 1) * (y - 1) * np.sin(x)
        val[..., 1] = x * (x - 1) * (y - 1) * np.sin(y)
        
        #val[..., 0] = x * (x - 1) * y * (y - 1)
        #val[..., 1] = x * (x - 1) * y * (y - 1)
        return val
        
    def is_dirichlet_boundary(self, p):
        x = p[..., 0]
        y = p[..., 1]
        flag = np.max(np.abs(y)) < 1e-13 
        return flag
    
def error(u, uh):
    e = u - uh
    emax = np.max(np.abs(e))
    return emax

def error_rel(uh1, uh2):
    val = uh2[0:uh1.shape[0]] - uh1
    return np.max(np.abs(val))

def print_error(Lam, H, E):
    for i in range(len(Lam)):
        print("---------------------Lam= {}---------------------".format(Lam[i]))
        #print("lam= ", Lam[i])
        print()
        for j in range(n):
            print("h= ", H[j])
            print("e=", E[i][j])
            #print("e_rel= ", E_rel[i,j])
            print()
        print()
    
def print_P(Lam, P):
    print("---------------------误差阶---------------------")
    for i in range(len(Lam)):
        print("lam= ", Lam[i])
        print("p= ", P[i])
        print()

# 返回 node 中是否为边界点的信息
def getIsBdNode(cr_node):
    is_BdNode = np.zeros(cr_node.shape[0], dtype=bool)
    for i in range(cr_node.shape[0]):
        #if ([0,0] == cr_node[i]).any() or ([1,1] == cr_node[i]).any():
        a = np.min(np.abs(cr_node[i] - np.array([0,0])))
        b = np.min(np.abs(cr_node[i] - np.array([1,1])))
        if a < 1e-13 or b < 1e-13:
            is_BdNode[i] = True
    return is_BdNode

i = 0
for lam in Lam:
    pde = PDE(1, lam)
    for j in range(n):
        mesh = pde.init_mesh(j)
        NN = mesh.number_of_nodes()
        NC = mesh.number_of_cells()
        #[NC,2] 剖分点及其编号(下标)
        node = mesh.entity('node') 
        #[NC,3] 剖分区间及其端点编号
        cell = mesh.entity('cell') 
        #[NC] 每个单元的面积
        cm = mesh.entity_measure()


        ##求解线性元导数
        node_cell_A = np.ones((NC, 3, 3), dtype=np.float64)
        #求解线性元导数的系数矩阵
        node_cell_A[:, :, 0:2] = node[cell]
        #print("cr_node_cell_A= ", cr_node_cell_A)
        #用于求解线性元的值
        # cr_glam_x_y_pre [NC, 3, 3]
        glam_x_y_pre = np.zeros((NC, 3, 3), dtype=np.float64)
        for k in range(NC):
            glam_x_y_pre[k, :, :] = solve(node_cell_A[k, :, :], np.diag(np.ones(3)))
        #print("cr_glam_x_y_pre= ", cr_glam_x_y_pre)
        #[NC,3,3]
        glam_x_y = np.copy(glam_x_y_pre)
        glam_x_y = glam_x_y[:, 0:2, :]
        #print("cr_glam_x_y= ", cr_glam_x_y)
        glam_x_y = glam_x_y.transpose((0,2,1))
        #print("cr_glam_x_y= ", cr_glam_x_y)
        
        #print("node= ", node)
        #print("cell= ", cell)
        ##[NC,3,2] 每个单元上基函数(三个)对x、y的偏导数
        #glam_x_y = mesh.grad_lambda()
        #print("glam_x_y= ", glam_x_y)
        #求 cr_phi_grad [NC,6(基函数),2(分量 x -> y),2(导数)]
        phi_grad = np.zeros((NC,6,2,2), dtype=np.float64)
        phi_grad[:, 0:5:2, 0, :] = glam_x_y
        phi_grad[:, 1:6:2, 1, :] = glam_x_y
        #print("phi_grad= ", phi_grad)
        
        # cr_phi_grad_t [NC,6,2,2]
        phi_grad_t = phi_grad.copy()
        phi_grad_t = phi_grad_t.transpose(0,1,3,2)
        #print("cr_phi_grad_t= ", cr_phi_grad_t)
        
        # cr_phi_epsilon [NC,6,2,2]
        phi_epsilon = (phi_grad + phi_grad_t) / 2
        #print("cr_phi_epsilon= ", cr_phi_epsilon)
        
        # cr_phi_div [NC, 6]
        #cr_phi_div = np.einsum("cmij -> cm", cr_phi_grad)
        phi_div = glam_x_y.copy()
        phi_div = phi_div.reshape(NC, 6)
        #print("cr_phi_div= ", cr_phi_div)

        #print("cm= ", cm)
        #print("cr_phi_sigma= ", cr_phi_sigma)
        #print("cr_phi_grad= ", cr_phi_grad)
        
        ## 刚度矩阵
        # A1 A2 [NC, 6, 6]
        A1 = np.einsum("cnij, cmij, c -> cnm", phi_epsilon, phi_epsilon , cm)
        A2 = np.einsum("cn, cm, c -> cnm", phi_div, phi_div, cm)
        #print("A1= ", A1)
        #print("A2= ", A2)
        
        
        cell_x_y = np.broadcast_to(cell[:,:,None], shape=(NC, 3, 2)).copy()
        cell_x_y[:,:,0] = 2 * cell_x_y[:,:,0]       #[NC,3] 三个节点x方向上基函数在总刚度矩阵的位置
        cell_x_y[:,:,1] = 2 * cell_x_y[:,:,1] + 1   #[NC,3] 三个节点y方向上基函数在总刚度矩阵的位置
        #print("cell= ", cell)
        #print("cell_x_y= ", cell_x_y)
        cell_x_y = cell_x_y.reshape(NC, 6)
        #print("cell_x_y= ", cell_x_y)
        I = np.broadcast_to(cell_x_y[:, :, None], shape=A1.shape)
        J = np.broadcast_to(cell_x_y[:, None, :], shape=A1.shape)
        #print("I= ", I)
        #print("J= ", J)

        #S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
        #M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
        #A = pde.mu * S + pde.lam * M
        #A = S + pde.lam * M
        A1 = csr_matrix((A1.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
        A2 = csr_matrix((A2.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
        A = 2 * pde.mu * A1 + pde.lam * A2
        #print("NN= ", NN)
        #print("A= ", np.linalg.matrix_rank(A.toarray()))
        #print("A= ", A.toarray())
        

        ##载荷向量

        from fealpy.quadrature import TriangleQuadrature

        NQ = 3
        Q = int(NQ * (NQ + 1) / 2) # 内积分点数
        qf = TriangleQuadrature(NQ)

        #bcs [Q,3]; WS[Q,]
        bcs,ws = qf.get_quadrature_points_and_weights()
        #print("bcs= ", bcs)
        #phi [Q,3,2]
        phi = np.broadcast_to(bcs[:, :, None], shape=[Q,3,2])
        #print("phi= ", phi)

        ##[Q,3,2(每个内积分节点两个基函数),2(每个基函数两个分量)]
        #phi_x_y = np.zeros((Q, 3, 2, 2), dtype=np.float64) 
        #phi_x_y[:, :, :, 0] = phi
        #phi_x_y[:, :, :, 1] = phi
        #phi_x_y[:, :, 0, 1] = 0
        #phi_x_y[:, :, 1, 0] = 0
        #phi_x_y = phi_x_y.reshape(Q,6,2)
        ##print("phi_x_y= ", phi_x_y)

        # node[cell] [NC,3(三个节点),2(每个节点的 x, y 坐标)]
        # ps [Q,NC,2]
        # 内积分点标准单元坐标转换为真实坐标
        ps = np.einsum('qi, cim->qcm', bcs, node[cell]) 
        #print("ps= ", ps)
        
        ##求CR元在各区间(积分节点)的值 #cr_ps_val
        phi_x_y  = np.zeros((Q, 3, 2, 2), dtype=np.float64)
        # ps_A [Q,NC,3]
        ps_A = np.ones((Q, NC, 3), dtype=np.float64)
        ps_A[:, :, 0:2] = ps
        #print("ps= ", ps)
        #print("ps_A= ", ps_A)
        #print("cr_glam_x_y_pre= ", cr_glam_x_y_pre)
        # cr_ps_val [Q,NC,3(3个基函数的值)]CR元在积分节点的值
        ps_val = np.einsum("qck, ckm -> qcm", ps_A, glam_x_y_pre)
        #print("cr_ps_val= ", cr_ps_val)
        
        
        ## 基函数在内积分节点的值
        # 每个基函数 [[\phi_i, 0][0, \phi_i]]
        # cr_ps_x_y [Q,NC,3,2(两个方向),2(两个分量)]
        ps_x_y = np.zeros((Q, NC, 3, 2, 2), dtype=np.float64)
        ps_x_y[:, :, :, 0, 0] = ps_val
        ps_x_y[:, :, :, 1, 1] = ps_val
        #print("cr_ps_x_y= ", cr_ps_x_y)
        ps_x_y = ps_x_y.reshape(Q, NC, 6, 2)
        #print("cr_ps_x_y= ", cr_ps_x_y)
        
        # val [Q,NC,2] 每个内积分的右端项的值
        val = pde.source(ps)
        #print("val= ", val)
        #bb [NC,6]
        #bb = np.einsum('q, qcj, qij, c->ci', ws, val, phi_x_y, cm)
        bb = np.einsum("q, qcj, qcij, c -> ci", ws, val, ps_x_y, cm)
        #print("cell= ", cell)
        #print("bb= ", bb)
        #print("cel_x_y= ", cell_x_y)
        bb = get_bb(pde, node, cell, cm)
        F = np.zeros(2 * NN)
        np.add.at(F, cell_x_y, bb)
        #print("F= ", F)


        ## 求解

        #isBdNode = mesh.ds.boundary_node_flag()
        isBdNode = getIsBdNode(node)
        isInterNode = ~isBdNode
        #print("isInterNode= ", isInterNode)
        isInterNode = np.broadcast_to(isInterNode[:, None], shape=(NN, 2))
        isInterNode = isInterNode.reshape(2 * NN)
        #print("isInterNode= ", isInterNode)

        uh = np.zeros((2 * NN), dtype=np.float64)
        uh[isInterNode] = spsolve(A[:, isInterNode][isInterNode], F[isInterNode])
        #print("uh= ", uh)
        uh = uh.reshape(NN, 2)
        #print("uh= ", uh)
        #print("F[isInterNode]= ", F[isInterNode])
        u = pde.solution(node)
        H[j] = np.sqrt(2 * cm[0])
        E[i][j] = error(u, uh)
        if j > 0:
            E_rel[i][j] = error_rel(uh_old, uh)
        uh_old = uh
    #print("i= ", i)
    i = i + 1

for i in range(len(Lam)):
    fig = plt.figure()
    plt.plot(np.log(H[1:]), np.log(E[i][1:]))
    plt.title("lam={}".format(Lam[i]))
    plt.xlabel("log(h)")
    plt.ylabel("log(e)")
    plt.savefig(fname="../image/tmp/Fem/elasticityFemLam_{}.png"
                .format(Lam[i]))
    plt.close(fig)
    
for i in range(len(Lam)):
    f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]),1)
    P[i] = f[0]

print_error(Lam, H, E)
print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.09490485121657727

h=  0.25
e= 0.02442414315497561

h=  0.125
e= 0.006214045568167001

h=  0.0625
e= 0.0015768788865023337


---------------------Lam= 10.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.15520641171385402

h=  0.25
e= 0.03699947583710775

h=  0.125
e= 0.009027831092635427

h=  0.0625
e= 0.002286790279529792


---------------------Lam= 100.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.18784827165028856

h=  0.25
e= 0.047714639311217846

h=  0.125
e= 0.01117390237929261

h=  0.0625
e= 0.002591372319443362


---------------------Lam= 1000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.19265708842038481

h=  0.25
e= 0.05041931506466196

h=  0.125
e= 0.012408572136687043

h=  0.0625
e= 0.002750081542911162


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.1931609802607684

h=  0.25
e= 0.05073928389772976

h=  0.125
e= 0.012637160818611624

h=  